Etape 1 : Extraction de la dixième colonne et des 1 000 premières réponses.

In [65]:
#import des bibliothèques et des fonctions
import os
import codecs
import csv
import re

#compteur
cpt=0

#déclaration des variables
rep=()

nlp = spacy.load('fr_core_news_sm')

#gérer les erreurs de noms de répertoire
try:
#chemin répertoires
    rep_in = "projet_tal_rep_in/" 
    rep_out = "projet_tal_rep_out/"
except FileNotFoundError:
    print ("Les chemins des répertoires ou leurs noms ne sont pas corrects")
else:
#lire tous les fichiers du répértoire
    for fichier_in in os.listdir(rep_in):
        if fichier_in.endswith(".csv"):#pour ne traiter que les fichiers qui se terminent par .csv
            fichier_out = re.sub("\.csv","_out.csv",fichier_in)
        
#ouverture des fichiers csv 
            reader= csv.reader(codecs.open(rep_in+fichier_in,"r",encoding="utf-8"),delimiter=",")#paramétrage du séparateur de colonnes
            writer= csv.writer(codecs.open(rep_out+fichier_out,"w",encoding="utf-8"),delimiter="\t")

#pour ne garder que la colonne K
            next(reader)#pour ne pas analyser la première ligne car elle contient la question
            for row in reader:
#un compteur pour ne traiter que les 1 000 premiers messages
                if cpt<1001:
                    rep=row[11]
                    cpt+=1
                else:
                    break
                if not rep=="": #pour ne pas garder les lignes vides
                    writer.writerow([row[11]])
            


Etape 2 : Nettoyage du fichier (tokenisation, suppression des accents et des cédilles, lemmatisation, mise en minuscule, suppression des mots vides, extraction des noms et adjectifs).


In [66]:
#import des bibliothèques et des fonctions
import spacy
from spacy.lang.fr.examples import sentences 
from spacy.lang.fr.stop_words import STOP_WORDS
import os
import codecs
import csv
import re

#choisir le nombre de colonne souhaité
col_pos = True
col_lemma = True
col_noStopWord = True

#gérer les erreurs de noms de répertoire
try:
#chemin répertoire
    rep_out2="projet_tal_rep_out2/"
except FileNotFoundError:
    print ("Le chemin du répertoire ou son nom n'est pas correct")
else:          
    for fichier_in2 in os.listdir(rep_out):
        if fichier_in2.endswith(".csv"):
            fichier_out2 = re.sub("\.csv","2.csv",fichier_in2)
#ouverture fichiers csv 
            reader= csv.reader(codecs.open(rep_out+fichier_in2,"r",encoding="utf-8"),delimiter="\t")
            writer= csv.writer(codecs.open(rep_out2+fichier_out2,"w",encoding="utf-8"),delimiter="\t")
        
            for row in reader:
                doc = nlp(row[0])
                rep_lemme=""#pour récupérer la réponses avec les mots lemmatisés + en minuscule
                rep_noStopw=""#pour récupérer la réponse sans les mots-vides
                rep_pos=""#pour récupérer les groupes nominaux (noms et adjectifs)
                
                #suppression des accents et des cédilles
                row[0]=re.sub(r"é","e",row[0])
                row[0]=re.sub(r"è","e",row[0])
                row[0]=re.sub(r"ê","e",row[0])
                row[0]=re.sub(r"à","a",row[0])
                row[0]=re.sub(r"â","a",row[0])
                row[0]=re.sub(r"î","i",row[0])
                row[0]=re.sub(r"ï","i",row[0])
                row[0]=re.sub(r"ë","e",row[0])
                row[0]=re.sub(r"ô","o",row[0])
                row[0]=re.sub(r"ü","u",row[0])
                row[0]=re.sub(r"û","u",row[0])
                row[0]=re.sub(r"ù","u",row[0])
                row[0]=re.sub(r"ç","c",row[0])
                
                doc = nlp(row[0])
                
                for token in doc:
                    if col_lemma:
#lemmatisation + mise minuscule
                        rep_lemme+=token.lemma_.lower()+" "
#enlever les mots-vides
                        if col_noStopWord:
                            if nlp.vocab[token.text].is_stop==False:
                                rep_noStopw+=token.lemma_+" "
#extraire les groupes nominaux (NOUN et ADJ)
                                if col_pos:
                                    if token.pos_ == "NOUN" or token.pos_=="ADJ":
                                        rep_pos+=token.lemma_.lower()+" "
                
              
                writer.writerow([row[0],rep_lemme,rep_noStopw,rep_pos])

Etape 3 : Calcule du tf-idf et clustering avec l'algorithme K-means.

In [68]:
#import des bibliothèques et des fonctions
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import numpy as np
import sys
import re
import os
import codecs
import csv

#gérer les erreurs de noms de répertoire
try:
#chemin répertoire
    rep_out3="projet_tal_rep_out3/"
except FileNotFoundError:
    print ("Le chemin du répertoire ou son nom n'est pas correct")
else:
    for fichier_in3 in os.listdir(rep_out2):
        if fichier_in3.endswith(".csv"):
            fichier_out3 = re.sub("2\.csv","3.txt",fichier_in3)#pour avoir des fichiers .txt en sortie
#ouverture fichiers csv 
            reader= csv.reader(codecs.open(rep_out2+fichier_in3,"r",encoding="utf-8"),delimiter="\t")#paramétrage du séparateur de colonnes
            writer=codecs.open(rep_out3+fichier_out3,"w", encoding="utf-8")
       
            content = []#une liste pour récupérer chaque réponse
            for row in reader:
                content.append(row[3])#ne récupère que la 4e colonne du document
        
#calcule le tf-idf
            tfidf = TfidfVectorizer(ngram_range=(1,1),max_features = 8000,token_pattern=r"(?u)\S\S+")#token_pattre = découpe au niveau des espaces
            textV=tfidf.fit(content)
            textV = tfidf.transform(content)
        
        
#modèle K-means
            model = KMeans(n_clusters=5, init='k-means++', max_iter=500, n_init=20)#charger le modèle de k-means
            model.fit(textV)

            prediction = model.fit_predict(textV)
            #print("\nVoici les principaux termes en fonction des clusters :")
            writer.write("Cluster\tRéponses d'internautes\n")
            for i in range(0, len(prediction)):
                writer.write(str(prediction[i])+"\t"+content[i]+"\n")
                #print (str(prediction[i])+" "+content[i])
            
#liste les mots de chaque clusters
            #print("\nVoici les principaux termes en fonction des clusters :")
            writer.write("\nClusters\tPrincipaux termes en fonction des clusters :\n")
            order_centroids = model.cluster_centers_.argsort()[:, ::-1]
            terms = tfidf.get_feature_names()
            for i in range(5):#choisir le nombre de clusters à afficher
                writer.write("\nCluster %d:" % i)
                print("\nCluster %d:" % i),
                for ind in order_centroids[i, :10]:#choisir le nombre de mots-clés à afficher
                    print(' %s' % terms[ind]),
                    writer.write('\t %s' % terms[ind])


Cluster 0:
 produit
 pesticide
 polluant
 agriculture
 entreprise
 consommation
 pollution
 industriel
 pays
 taxe

Cluster 1:
 vehicule
 electrique
 niveau
 voiture
 polluant
 mondial
 diesel
 entreprise
 pollution
 pourcent

Cluster 2:
 pollueur
 pays
 gros
 payeur
 avion
 grand
 sanction
 politique
 etat
 paquebot

Cluster 3:
 energie
 renouvelable
 fossile
 nucleaire
 centrale
 propre
 solaire
 reduire
 consommation
 production

Cluster 4:
 transport
 commun
 ville
 voiture
 camion
 grand
 produit
 routier
 deplacement
 public


Evaluation des résultats

In [69]:
#import des bibliothèques et des fonctions
import codecs

#gérer les erreurs de noms de fichiers
try:
#chemin fichiers
    fichier_in3=codecs.open("LA_TRANSITION_ECOLOGIQUE_COPIE.txt","r",encoding="utf-8")
    fichier_out=codecs.open("LA_TRANSITION_ECOLOGIQUE_EVAL.txt","w",encoding="utf-8")
except NameError:
    print ("Le nom du fichier n'est pas correct")
else:
    fichier_out.write("Voici les résultats de l'évaluation des clusters :\n")

#calcule du rappel
    def rappel(mot_0,mot_1,mot_2,mot_3,mot_4,fichier):
        sentence=[]
        vp=0
        fn=0
        for ligne in fichier_in3:
            if len(ligne) > 1:
                ligne=ligne.split("\t")
                if mot_0 in ligne[0] and mot_0 in ligne[1]:
                    vp+=1
                elif mot_1 in ligne[0] and mot_1 in ligne[1]:
                    vp+=1
                elif mot_2 in ligne[0] and mot_2 in ligne[1]:
                    vp+=1
                elif mot_3 in ligne[0] and mot_3 in ligne[1]:
                    vp+=1
                elif mot_4 in ligne[0] and mot_4 in ligne[1]:
                    vp+=1
                elif mot_0 in ligne[0] and not mot_0 in ligne[1]:
                    fn+=1
                elif mot_1 in ligne[0] and not mot_1 in ligne[1]:
                    fn+=1
                elif mot_2 in ligne[0] and not mot_2 in ligne[1]:
                    fn+=1
                elif mot_3 in ligne[0] and not mot_3 in ligne[1]:
                    fn+=1
                elif mot_4 in ligne[0] and not mot_4 in ligne[1]:
                    fn+=1
        rep_rap=vp/(vp+fn)
        return rep_rap
    
    rep_rappel = rappel(mot_0="0",mot_1="1", mot_2="2",mot_3="3", mot_4="4",fichier=fichier_in3)
    fichier_out.write("Le rappel est de "+str(rep_rappel)+"\n")
    print("Le rappel est de "+str(rep_rappel)+"\n")
    
    fichier_in3=codecs.open("LA_TRANSITION_ECOLOGIQUE_COPIE.txt","r",encoding="utf-8")

#calcule de la précision
    def precision(mot_0,mot_1,mot_2,mot_3,mot_4,fichier):
        sentence=[]
        vp=0
        fp=0
        for ligne in fichier_in3:
            if len(ligne) > 1:
                ligne=ligne.split("\t")
                if mot_0 in ligne[0] and mot_0 in ligne[1]:
                    vp+=1
                elif mot_1 in ligne[0] and mot_1 in ligne[1]:
                    vp+=1
                elif mot_2 in ligne[0] and mot_2 in ligne[1]:
                    vp+=1
                elif mot_3 in ligne[0] and mot_3 in ligne[1]:
                    vp+=1
                elif mot_4 in ligne[0] and mot_4 in ligne[1]:
                    vp+=1
                elif mot_0 in ligne[1] and not mot_0 in ligne[0]:
                    fp+=1
                elif mot_1 in ligne[1] and not mot_1 in ligne[0]:
                    fp+=1
                elif mot_2 in ligne[1] and not mot_2 in ligne[0]:
                    fp+=1
                elif mot_3 in ligne[1] and not mot_3 in ligne[0]:
                    fp+=1
                elif mot_4 in ligne[1] and not mot_4 in ligne[0]:
                    fp+=1
 
        rep_pre=vp/(vp+fp)
        return rep_pre
    
    rep_precision = precision(mot_0="0",mot_1="1", mot_2="2",mot_3="3", mot_4="4",fichier=fichier_in3)
    fichier_out.write("La précision est de "+str(rep_precision)+"\n")
    print("La précision est de "+str(rep_precision)+"\n")
    
#calcule de la f-mesure
    def fmesure(fichier,rappel,precision):
        rep=(2*rappel*precision)/(rappel+precision)
        print("La f-mesure est de ",rep)
        fichier_out.write("la f-mesure est de "+str(rep))
    
    fmesure(fichier=fichier_in3,rappel=rep_rappel,precision=rep_precision)

    fichier_in3.close()
    fichier_out.close()

Le rappel est de 0.75

La précision est de 0.7241379310344828

La f-mesure est de  0.736842105263158
